# A. Heston Stochastic Volatility 
* Path dependent payoffs
* Discontinuous payoffs (digitals, barriers)
* Highly convex products (gamma, volga)
* Non-gaussian dynamics
* Includes both european and asian digitals

In [19]:
import numpy as np
import pandas as pd
from dataclasses import dataclass


# Heston MC simulation + pricing
- truncation euler scheme

In [20]:
@dataclass
class HestonModel:
    S0: float
    v0: float
    r: float
    T: float
    steps: int
    Npaths: int
    kappa: float
    theta: float
    xi: float
    rho: float

    def simulate_paths(self, S0=None, v0=None, r=None, seed=None):
        """Full truncation Euler scheme for Heston."""
        if S0 is None: S0 = self.S0
        if v0 is None: v0 = self.v0
        if r  is None: r  = self.r
        if seed is not None:
            np.random.seed(seed)

        dt = self.T / self.steps
        S = np.zeros((self.Npaths, self.steps + 1))
        v = np.zeros((self.Npaths, self.steps + 1))

        S[:, 0] = S0
        v[:, 0] = v0

        for t in range(self.steps):
            z1 = np.random.normal(size=self.Npaths)
            z2 = np.random.normal(size=self.Npaths)
            z2 = self.rho*z1 + np.sqrt(1 - self.rho**2)*z2

            v_pos = np.maximum(v[:, t], 0)
            v[:, t+1] = (v[:, t] 
                         + self.kappa*(self.theta - v_pos)*dt
                         + self.xi*np.sqrt(v_pos)*np.sqrt(dt)*z2)
            v[:, t+1] = np.maximum(v[:, t+1], 0)

            S[:, t+1] = S[:, t] * np.exp(
                (r - 0.5*v_pos)*dt + np.sqrt(v_pos*dt)*z1
            )

        return S, v

    def price(self, payoff_fn, S0=None, v0=None, r=None, **kwargs):
        """Monte Carlo pricing wrapper."""
        S, _ = self.simulate_paths(S0=S0, v0=v0, r=r)
        payoff = payoff_fn(S, **kwargs)
        discounted = np.exp(-self.r*self.T)*payoff.mean()
        return discounted, payoff


# Exotic Payoffs Module

In [21]:
class Payoffs:

    @staticmethod
    def asian_call(S, K):
        avg = S.mean(axis=1)
        return np.maximum(avg - K, 0.0)

    @staticmethod
    def digital_call(S, K):
        ST = S[:, -1]
        return (ST > K).astype(float)

    @staticmethod
    def digital_asian(S, K):
        avg = S.mean(axis=1)
        return (avg > K).astype(float)

    @staticmethod
    def barrier_up_out_call(S, K, B):
        barrier_hit = (S.max(axis=1) >= B)
        return np.where(barrier_hit, 0.0, np.maximum(S[:, -1] - K, 0.0))


FD Greeks Engine
* reusable greeks class for any payoff

In [22]:
class Greeks:

    @staticmethod
    def delta(model, payoff_fn, h=0.5, **kwargs):
        p_up, _ = model.price(payoff_fn, S0=model.S0 + h, **kwargs)
        p_dn, _ = model.price(payoff_fn, S0=model.S0 - h, **kwargs)
        return (p_up - p_dn) / (2*h)

    @staticmethod
    def gamma(model, payoff_fn, h=0.5, **kwargs):
        p_up, _ = model.price(payoff_fn, S0=model.S0 + h, **kwargs)
        p_0,  _ = model.price(payoff_fn, S0=model.S0, **kwargs)
        p_dn, _ = model.price(payoff_fn, S0=model.S0 - h, **kwargs)
        return (p_up - 2*p_0 + p_dn) / (h*h)

    @staticmethod
    def vega(model, payoff_fn, h=0.002, **kwargs):
        p_up, _ = model.price(payoff_fn, v0=model.v0 + h, **kwargs)
        p_dn, _ = model.price(payoff_fn, v0=model.v0 - h, **kwargs)
        return (p_up - p_dn) / (2*h)


# P&L Explain Engine
* risk-based and step revaluation

In [23]:
class PnLExplain:

    @staticmethod
    def risk_based(delta, gamma, vega, dS, dv):
        return delta*dS + 0.5*gamma*dS*dS + vega*dv

    @staticmethod
    def step_reval(model, payoff_fn, S1, v1, r1, **kwargs):
        # SOD price
        P0, _ = model.price(payoff_fn, **kwargs)

        # Step 1: Spot shift
        P1, _ = model.price(payoff_fn, S0=S1, **kwargs)
        SR_spot = P1 - P0

        # Step 2: Volatility shift
        P2, _ = model.price(payoff_fn, S0=S1, v0=v1, **kwargs)
        SR_vol = P2 - P1

        # Step 3: Rate shift
        P3, _ = model.price(payoff_fn, S0=S1, v0=v1, r=r1, **kwargs)
        SR_rate = P3 - P2

        return SR_spot + SR_vol + SR_rate


# Define Market and Heston Parameters

In [24]:
# Market
S0 = 100
v0 = 0.04
r0 = 0.02
T = 1.0
steps = 252
Npaths = 200_000

# Heston params
kappa = 1.5
theta = 0.04
xi    = 0.5
rho   = -0.7

model = HestonModel(S0, v0, r0, T, steps, Npaths, kappa, theta, xi, rho)


# Price Portfolio of Exotics at SOD

In [25]:
K = 100
B = 130

asian_SOD, _   = model.price(Payoffs.asian_call,   K=K)
digital_SOD, _ = model.price(Payoffs.digital_call, K=K)
d_as_SOD, _    = model.price(Payoffs.digital_asian, K=K)
bar_SOD, _     = model.price(Payoffs.barrier_up_out_call, K=K, B=B)


# Greeks at SOD

In [26]:
asian_delta = Greeks.delta(model, Payoffs.asian_call, K=K)
asian_gamma = Greeks.gamma(model, Payoffs.asian_call, K=K)
asian_vega  = Greeks.vega(model, Payoffs.asian_call, K=K)

digital_delta = Greeks.delta(model, Payoffs.digital_call, K=K)
digital_gamma = Greeks.gamma(model, Payoffs.digital_call, K=K)
digital_vega  = Greeks.vega(model, Payoffs.digital_call, K=K)

das_delta = Greeks.delta(model, Payoffs.digital_asian, K=K)
das_gamma = Greeks.gamma(model, Payoffs.digital_asian, K=K)
das_vega  = Greeks.vega(model, Payoffs.digital_asian, K=K)

bar_delta = Greeks.delta(model, Payoffs.barrier_up_out_call, K=K, B=B)
bar_gamma = Greeks.gamma(model, Payoffs.barrier_up_out_call, K=K, B=B)
bar_vega  = Greeks.vega(model, Payoffs.barrier_up_out_call, K=K, B=B)


# Simulate Observed next-day Market Moves

In [27]:
dS  = +1.3
dv0 = +0.003
dr  = -0.002

S1  = S0 + dS
v1  = v0 + dv0
r1  = r0 + dr


# Price Portfolio at COB

In [28]:
asian_COB, _   = model.price(Payoffs.asian_call,   S0=S1, v0=v1, r=r1, K=K)
digital_COB, _ = model.price(Payoffs.digital_call, S0=S1, v0=v1, r=r1, K=K)
d_as_COB, _    = model.price(Payoffs.digital_asian, S0=S1, v0=v1, r=r1, K=K)
bar_COB, _     = model.price(Payoffs.barrier_up_out_call, S0=S1, v0=v1, r=r1, K=K, B=B)

PnL_actual = ((asian_COB - asian_SOD)
             + (digital_COB - digital_SOD)
             + (d_as_COB - d_as_SOD)
             + (bar_COB - bar_SOD))


# Risk-based P&L Explain

In [29]:
PnL_rb = (
    PnLExplain.risk_based(asian_delta, asian_gamma, asian_vega, dS, dv0)
  + PnLExplain.risk_based(digital_delta, digital_gamma, digital_vega, dS, dv0)
  + PnLExplain.risk_based(das_delta, das_gamma, das_vega, dS, dv0)
  + PnLExplain.risk_based(bar_delta, bar_gamma, bar_vega, dS, dv0)
)

unexplained_rb = PnL_actual - PnL_rb


# Step Revaluation P&L Explain

In [30]:
PnL_sr = (
    PnLExplain.step_reval(model, Payoffs.asian_call, S1, v1, r1, K=K)
  + PnLExplain.step_reval(model, Payoffs.digital_call, S1, v1, r1, K=K)
  + PnLExplain.step_reval(model, Payoffs.digital_asian, S1, v1, r1, K=K)
  + PnLExplain.step_reval(model, Payoffs.barrier_up_out_call, S1, v1, r1, K=K, B=B)
)

unexplained_sr = PnL_actual - PnL_sr


In [31]:
pd.DataFrame({
    "Actual P&L": [PnL_actual],
    "Risk-Based P&L": [PnL_rb],
    "RB Unexplained": [unexplained_rb],
    "Step-Reval P&L": [PnL_sr],
    "SR Unexplained": [unexplained_sr]
})


,Actual P&L,Risk-Based P&L,RB Unexplained,Step-Reval P&L,SR Unexplained
0,1.321054,1.844339,-0.523284,1.274928,0.046126
